In [ ]:
import numpy as np
import pandas as pd
import datetime
import pytz

In [ ]:
#Bokeh
from bokeh.io import show
from bokeh.plotting import figure
from  big_data_tools.bokeh_tools.bar import bar 
from  big_data_tools.bokeh_tools.histogram import hist 
from  big_data_tools.bokeh_tools.box_plot import box_plot 
from  big_data_tools.bokeh_tools.prob_plot import prob_plot 
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter
from bokeh.models import DatetimeTickFormatter

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import ensemble

In [ ]:
def get_data(path):
    def convert_date(x):
        return datetime.datetime.strptime(x[0:8], '%Y%m%d').replace(tzinfo=pytz.utc)
    d= pd.read_csv(path)
    d['date'] = d['date'].map(convert_date)
    return d
housing_data = get_data('/Users/a6002538/Downloads/kc_house_data.csv')
housing_data.head()

In [ ]:
housing_data.describe()

In [ ]:
def make_histogram(data):
    return hist(data)
p_housing_hist = make_histogram(housing_data['bedrooms'])

In [ ]:
def make_floors_price_scatter(data):
    p = figure()
    p.circle(x=data.floors, y = data.price)
    return p
p_floors_price = make_floors_price_scatter(housing_data)

In [ ]:
reg = LinearRegression()
labels = housing_data['price']
conv_dates = [1 if values.year == 2014 else 0 for values in housing_data.date ]
housing_data['date'] = conv_dates
train1 = housing_data.drop(['id', 'price'],axis=1)

In [ ]:
def make_pred_graphs(true_data, pred):
    p_hist = hist(true_data - pred)
    p_scatter = figure()
    p_scatter.circle(x = true_data, y = pred)
    max_y = max(max(pred), max(true_data))
    p_scatter.line(x = [0, max_y], y = [0, max_y])
    return p_hist, p_scatter

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(train1 , labels , test_size = 0.10,random_state =2)
reg.fit(x_train,y_train)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
reg.score(x_test,y_test)
#print(reg.coef_)
pred1 = reg.predict(train1)
housing_data['price'] - pred1
p_pred_hist, p_pred_scatter = make_pred_graphs(housing_data['price'], pred1)



In [80]:
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')
clf.fit(x_train, y_train)
clf.score(x_test,y_test)



In [97]:
params = clf.get_params()
print(dir(clf))
print(clf.loss_())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_raw_predict', '_raw_predict_init', '_required_parameters', '_resize_state', '_rng', '_staged_raw_predict', '_validate_estimator', '_validate_y', 'alpha', 'apply', 'criterion', 'estimators_', 'feature_importances_', 'fit', 'get_params', 'init', 'init_', 'learning_rate

TypeError: __call__() missing 2 required positional arguments: 'y' and 'raw_predictions'

In [91]:
pred2 = clf.predict(train1)
#[x for x in pred2]
p_pred_hist2, p_pred_scatter2 = make_pred_graphs(housing_data['price'], [x for x in pred2])


(2150000.0, 1131671.2507917702, 1018328.7492082298)
(2850000.0, 1746700.7244163202, 1103299.2755836798)
(2110000.0, 1062502.561884071, 1047497.438115929)


In [85]:
#all graphs
grid = gridplot([p_housing_hist,
    p_floors_price, p_pred_hist,
    p_pred_scatter, p_pred_hist2, 
                 p_pred_scatter2,
    ],
    ncols = 2)
show(grid)